# PromptTemplate 的使用 (Usage of PromptTemplate)

1、PromptTemplate 如何获取实例 (How to instantiate PromptTemplate)

2、两种特殊结构的使用 (Use of two special structures)
    (部分提示词模板的使用、组合提示词的使用)
    (Partial Prompt Template Usage, Combined Prompt Usage)

3、给变量赋值的两种方式: format() / invoke() (Two ways to assign values to variables)

4、结合大模型的使用 (Combining usage with a Large Language Model)


## 1.PromptTemplate 如何获取实例
方式1: 使用构造方法的方式

In [ ]:
from langchain_core.prompts import PromptTemplate

#1. 创建promptTemplate的实例
# # 参数中必须指明: input_variables、template
prompt_template = PromptTemplate(
    template="你是一个{role}, 你的名字叫{name}",
    input_variables=["role", "name"],
)

# print(prompt_template)

# #2、填充实例中的变量，暂时使用 format()
# 使用 .format() 方法，以关键字参数的形式传入变量的值
prompt = prompt_template.format(role="人工智能专家", name="小智")

# 打印填充后的完整提示词字符串
print(prompt)

方法2：form——template(): 推荐

In [ ]:
from langchain_core.prompts import PromptTemplate

# #1、创建 PromptTemplate 的实例
# 这种方法更简洁，它会自动从 template 字符串中识别出 input_variables
prompt_template = PromptTemplate.from_template(template="你是一个{role}, 你的名字叫{name}")

print(prompt_template)
# 运行 print(prompt_template) 会显示：
# input_variables=['role', 'name'] template='你是一个{role}, 你的名字叫{name}'

# #2、填充实例中的变量，暂时使用 format()
prompt = prompt_template.format(role="人工智能专家", name="小智")

print(prompt)
# 运行 print(prompt) 会显示：
# 你是一个人工智能专家, 你的名字叫小智

In [ ]:
#如果提示词模板中不包含变量，则调用 format() 时，不需要传入参数！

# #1. 导入相关的包
from langchain_core.prompts import PromptTemplate

# # 2. 定义提示词模板对象
text = """
Tell me a joke
"""

# 使用 from_template 创建实例
# 因为 text 中不包含任何花括号 {} 变量，所以 input_variables 将为空列表
prompt_template = PromptTemplate.from_template(text)

# # 3. 默认使用 f-string 进行格式化 (返回格式好的字符串)
# 因为没有变量需要填充，format() 直接返回 template 文本
prompt = prompt_template.format()

print(prompt)

## 2、两种特殊结构的使用 (部分提示词模板的使用、组合提示词的使用)

### 2.1 部分提示词模板的使用 (重点)

In [ ]:
# 导入所需的包
from langchain_core.prompts import PromptTemplate

# # 定义多变量模板
# 注意：在 LangChain 0.2.0+ 版本中，PromptTemplate 已经移到了 langchain_core.prompts
# 确保你的导入是 from langchain_core.prompts import PromptTemplate
template = PromptTemplate.from_template(
    template="请评价{product}的优缺点。包括{aspect1}和{aspect2}。",
    # 在实例化时，直接通过 partial_variables 预先锁定 aspect1 的值
    partial_variables={"aspect1": "电池续航"}
)

# # 使用模板生成提示词
# 此时，我们只需要为 product 和 aspect2 赋值
prompt_1 = template.format(product="智能手机", aspect2="拍照质量")

print(prompt_1)

In [ ]:
from langchain_core.prompts import PromptTemplate

# # 定义多变量模板
# 注意：这里使用的是 PromptTemplate() 构造函数，所以需要明确指定 input_variables
template = PromptTemplate(
    template="请评价{product}的优缺点。包括{aspect1}和{aspect2}。",
    input_variables=["product", "aspect1", "aspect2"],
    # 在实例化时，通过 partial_variables 预先锁定 aspect1 和 aspect2 的值
    partial_variables={"aspect1": "电池续航", "aspect2": "拍照质量"}
)

# # 使用模板生成提示词
## partial_variables就是设置默认值，可以覆盖
prompt_1 = template.format(product="智能手机", aspect1="电池续航11", aspect2="拍照质量22")

print("提示词:", prompt_1)

方式2：调用方法partial()

In [ ]:
from langchain_core.prompts import PromptTemplate

# # 定义多变量模板
# 1. 创建原始的 Prompt Template
# 2. 紧接着在构造函数后面链式调用 .partial() 方法，创建并返回一个新的模板对象
template = PromptTemplate(
    template="请评价{product}的优缺点。包括{aspect1}和{aspect2}。",
    input_variables=["product", "aspect1", "aspect2"],
    # partial_variables 这一行被注释掉了，表示这里主要演示 .partial() 方法
    # # partial_variables={"aspect1": "电池续航", "aspect2": "拍照质量"}
).partial(aspect1="电池续航", aspect2="拍照质量") # 注意这里没有变量接收，模板被修改

# 警告/说明：
# 尽管代码中变量名仍为 'template'，但它现在持有的是 .partial() 返回的 **新模板**，
# 这个新模板的 input_variables 只包含 ['product']。

# # 使用模板生成提示词
# 3. 只需为新模板中剩余的变量赋值
prompt_1 = template.format(product="智能手机")

print("提示词:", prompt_1)

2.2 组合提示词的使用 (了解)

In [ ]:
from langchain_core.prompts import PromptTemplate

# 组合提示词模板
# 它将一个 PromptTemplate 对象、一个固定的字符串、以及另一个带有变量的字符串拼接起来
template = (
    PromptTemplate.from_template(template = "Tell me a joke about {topic}")
    + ", make it funny" # 拼接一个固定字符串
    + "\nand in {language}" # 拼接另一个含有新变量的字符串
)

# 打印 template 实例的输入变量，你会发现它自动包含了 {topic} 和 {language}
# print(template.input_variables) # ['topic', 'language']

# 格式化组合提示词
# 传入所有必需的变量
prompt = template.format(topic="sports", language="spanish")

print(prompt)

## 3、给变量赋值的两种方式: format() / invoke()

format()：参数部分: 给变量赋值; | 返回值: str类型

invoke()：参数部分: 使用的是字典; | 返回值: PromptValue类型                   返回值:

In [ ]:
from langchain_core.prompts import PromptTemplate

# # 定义多变量模板
# 使用 from_template 简洁创建模板，它会自动识别 {product}、{aspect1} 和 {aspect2}
template = PromptTemplate.from_template(
    template="请评价{product}的优缺点。包括{aspect1}和{aspect2}。"
)

# # 使用模板生成提示词
# 使用 .format() 填充所有变量
prompt_1 = template.format(product="智能手机", aspect1="电池续航", aspect2="拍照质量")

print(prompt_1)
print(type(prompt_1))

调用invoke()

In [ ]:
from langchain_core.prompts import PromptTemplate

# # 定义多变量模板
# 使用 from_template 简洁创建模板，它会自动识别 {product}、{aspect1} 和 {aspect2}
template = PromptTemplate.from_template(
    template="请评价{product}的优缺点。包括{aspect1}和{aspect2}。"
)

# # 使用模板生成提示词
# 使用 .format() 填充所有变量
prompt_1 = template.invoke(input= {"product":"智能手机", "aspect1":"电池续航", "aspect2":"拍照质量"})

print(prompt_1)
print(type(prompt_1))

## 4、结合大模型的使用


In [ ]:
from langchain_openai import ChatOpenAI
import os
import dotenv

#加载配置文件
dotenv.load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

chat_model = ChatOpenAI(
    model = "gpt-4o-mini",
)

# 生成提示词模板
template = PromptTemplate.from_template(
    template="请评价{product}的优缺点，包括{aspect1}和{aspect2}。"
)

# 给模板的变量赋值
prompt = template.invoke(input={"product": "智能手机", "aspect1": "电池续航", "aspect2": "拍照质量"})

# 调用大模型，将提示词传入
response = chat_model.invoke(prompt)

print(response)
print(type(response))